In [10]:
'''
    Copyright 2022 by Michał Stolarz <michal.stolarz@h-brs.de>

    This file is part of migrave_personalised_behaviour_model.
    It is used for calculating estimated engagement for each user for each seocond of an interaction.

    migrave_personalised_behaviour_model is free software: you can redistribute it and/or modify
    it under the terms of the GNU Affero General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.
    migrave_personalised_behaviour_model is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU Affero General Public License for more details.
    You should have received a copy of the GNU Affero General Public License
    along with migrave_personalised_behaviour_model. If not, see <http://www.gnu.org/licenses/>.
'''

from person_state_estimation import PersonStateEstimation
import pandas as pd
import os
import yaml
import numpy as np
import matplotlib.pyplot as plt

In [11]:
def parse_yaml_config(config_file):
    if config_file and os.path.isfile(config_file):
        configs = {}
        with open(config_file, 'r') as infile:
            configs = yaml.safe_load(infile)

        return configs
    else:
        print("Config not found or not given")

In [ ]:
# Loading the configuration file
config = parse_yaml_config("person_state_estimation_config.yaml")
path = ""
person_state_estimator = PersonStateEstimation(config, path)
person_state_estimator.load_classifier("models/migrave_engagement_model_xgboost.joblib")

In [13]:
# Calculating estimated engagement for data from the tablet perspective
df = pd.read_csv("data/openface_tablet.csv")

df_cpy = df.copy(deep=True)
df_cpy.drop(['participant_id', 'secs', 'nsecs'], inplace=True, axis=1)

df = df[['participant_id', 'secs']]
df['engagement'] = 0

eng_val, score_val = person_state_estimator.estimate_engagement(df_cpy)
df['engagement'] = eng_val

tablet_df = df.groupby(["participant_id", "secs", "engagement"]).size().unstack(fill_value=0)

tablet_df['eng_tab'] = None

for idx,row in tablet_df.iterrows():
    val_sum = row[-1] + row[1]
    tablet_df.loc[idx, 'eng_tab'] = -1*row[-1]/val_sum + 1*row[1]/val_sum

In [14]:
# Calculating estimated engagement for data from the robot head perspective
df1 = pd.read_csv("data/openface_qt.csv")

df1_cpy = df1.copy(deep=True)
df1_cpy.drop(['participant_id', 'secs', 'nsecs'], inplace=True, axis=1)

df1 = df1[['participant_id', 'secs']]
df1['engagement'] = 0

eng_val, score_val = person_state_estimator.estimate_engagement(df1_cpy)
df1['engagement'] = eng_val

qt_df = df1.groupby(["participant_id", "secs", "engagement"]).size().unstack(fill_value=0)

qt_df['eng_qt'] = None

for idx,row in qt_df.iterrows():
    val_sum = row[-1] + row[1]
    qt_df.loc[idx, 'eng_qt'] = -1*row[-1]/val_sum + 1*row[1]/val_sum

In [15]:
# Plotting engagement evolution for one of the users
eng_df = qt_df
qt = eng_df.groupby("participant_id").get_group('1CZ1CL1P')['eng_qt']

In [ ]:
plt.figure(figsize=(20, 10))
ax1 = qt.plot(label='QT')
h1, l1 = ax1.get_legend_handles_labels()
plt.legend(h1, l1, loc=2)
plt.ylabel("Engagement score")
plt.xlabel("Time [s]")
plt.grid()

In [8]:
# Dropping expected engagement estimates from the tablet, as the model was trained on the data from the robot head perspective, thus a model has big misclassification rate
eng_df = eng_df[["eng_qt"]]
eng_df_fin = eng_df.reset_index(level=["participant_id", "secs"])

In [9]:
eng_df_fin.to_csv("output/eng_final.csv", header=True, index=None, sep=',', float_format='%10.4f', mode='w')